## Connecting to the GeoSpock SQL Access cluster
There are many ways to connect Jupyter Notebooks to your GeoSpock presto cluster for SQL access. We give three examples
here, with a username and password read from a local file, and the host string parameterised. There is also a quick
query to run with each to demonstrate that results are returned as expected.

 - Using presto-python-client
 - Using sqlalchemy
 - Using ipython-sql

### Setting up and reading credentials
Please update the host (and port if necessary) in line with your deployment, and save your **GeoSpock DB** credentials
as a username and password separated by a space in a `.geospockcredentials.txt` file in your home directory.

In [ ]:
from os.path import expanduser
import pandas as pd

home = expanduser("~")
with open(home + "/.geospockcredentials.txt", "r") as file:
    x = file.readlines()

[username,password] = x[0].split(" ")
host = "<your deploymeny host>"
port = 8446

### Connecting using presto-python-client
An example using the `presto-python-client` package.

In [ ]:
import prestodb.dbapi as presto
import prestodb

conn = presto.Connection(
    host=host,
    port=port,
    user=username,
    http_scheme='https',
    auth=prestodb.auth.BasicAuthentication(username, password))
cur = conn.cursor()

cur.execute('SELECT * FROM geospock.default.tweet LIMIT 10')
result = cur.fetchall()

df = pd.DataFrame(sorted(result, key=lambda x: x[2], reverse=True))
display(df)

### Connecting using sqlalchemy
An example using the `sqlalchemy` package.

In [ ]:
from sqlalchemy.engine import create_engine

# Presto
engine = create_engine('presto://{username}:{password}@{host}:{port}'.format(username=username, password=password, host=host, port=port),
                      connect_args={'protocol': 'https'})

df = pd.read_sql('SELECT * FROM geospock.default.tweet LIMIT 10', engine)
display(df)

### Connecting using ipython-sql
An example using the `ipython-sql` package.

In [ ]:
%load_ext sql

%config SqlMagic.autocommit=False

In [ ]:
%sql presto://{username}:{password}@{host}:{port}/geospock?protocol=https

%sql SELECT * FROM geospock.default.tweet LIMIT 10